In [1]:
# import necessary packages
import numpy as np
import scipy as sci

In [2]:
# define number of qubits
N = 3

In [3]:
# create qubit register with all states being at equal probability of 0 state with amplitude 1/sqrt(2^N)
def qubit_register(N):
    
    # initialize column vector
    psi = 1 / np.sqrt(2**N) * np.zeros((2**N, 1), dtype=complex)
    psi[0] = 1
    
    return psi

In [4]:
# measure probability of each basis state
def measure_Sz(state):
    
    probabilities = np.abs(state)**2
    
    # select a basis state randomly according to the probabilities
    measurement = np.random.choice(np.arange(len(state)), p=probabilities.ravel())
    
    return measurement

In [5]:
# perform Hadamard transformation for any N
def hadamard_transform(N):
    # define the initial Hadamard matrix
    hadamard_initial = 1 / np.sqrt(2) * np.array([[1, 1], [1, -1]])

    # number of iterations for the transformation
    num_iterations = N - 1

    # initialize the resulting tensor product matrix
    hadamard_result = hadamard_initial

    # perform tensor product iteratively
    for i in range(num_iterations):
        hadamard_result = np.kron(hadamard_initial, hadamard_result)
        
    return hadamard_result

In [6]:
# print the register
psi = qubit_register(N)
print("The qubit register is:\n", psi)


The qubit register is:
 [[1.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]]


In [7]:
# print the basis state
measurement = measure_Sz(psi)
print("The measured basis state is:\n|",np.binary_repr(measurement, width = N),">")

The measured basis state is:
| 000 >


In [8]:
# print the Hadamard transformed
hadamard = hadamard_transform(N)
print("The resulting Hadamard is:\n", hadamard)

The resulting Hadamard is:
 [[ 0.35355339  0.35355339  0.35355339  0.35355339  0.35355339  0.35355339
   0.35355339  0.35355339]
 [ 0.35355339 -0.35355339  0.35355339 -0.35355339  0.35355339 -0.35355339
   0.35355339 -0.35355339]
 [ 0.35355339  0.35355339 -0.35355339 -0.35355339  0.35355339  0.35355339
  -0.35355339 -0.35355339]
 [ 0.35355339 -0.35355339 -0.35355339  0.35355339  0.35355339 -0.35355339
  -0.35355339  0.35355339]
 [ 0.35355339  0.35355339  0.35355339  0.35355339 -0.35355339 -0.35355339
  -0.35355339 -0.35355339]
 [ 0.35355339 -0.35355339  0.35355339 -0.35355339 -0.35355339  0.35355339
  -0.35355339  0.35355339]
 [ 0.35355339  0.35355339 -0.35355339 -0.35355339 -0.35355339 -0.35355339
   0.35355339  0.35355339]
 [ 0.35355339 -0.35355339 -0.35355339  0.35355339 -0.35355339  0.35355339
   0.35355339 -0.35355339]]


In [9]:
# checks the Hadamard is correct by dividing by the amplitude
hadamard_check = hadamard / (1 / (2**(N/2)))
print("The checked Hadamard without the amplitude is:\n", hadamard_check)

The checked Hadamard without the amplitude is:
 [[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1. -1.  1. -1.  1. -1.  1. -1.]
 [ 1.  1. -1. -1.  1.  1. -1. -1.]
 [ 1. -1. -1.  1.  1. -1. -1.  1.]
 [ 1.  1.  1.  1. -1. -1. -1. -1.]
 [ 1. -1.  1. -1. -1.  1. -1.  1.]
 [ 1.  1. -1. -1. -1. -1.  1.  1.]
 [ 1. -1. -1.  1. -1.  1.  1. -1.]]
